In [1]:
from os import listdir
from time import time

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow import math
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from utils import load_dataset, train

In [2]:
from tensorflow.config import list_physical_devices

assert len(list_physical_devices('GPU'))

In [3]:
def schedule(epoch, lr) -> float:
    if epoch >= 200 and epoch % 25 == 0:
        lr = lr * math.exp(-0.1)
#         print(f'Epoch {epoch}\tNew LR: {lr}')
    return lr

In [4]:
def plot_roc(y_true, y_score, boundary: int):
    y = np.where(y > boundary, 0, 1)
    pred = np.where(pred > boundary, 0, 1)

    fpr, tpr, thresholds = roc_curve(y, pred)

    plt.title(f'ROC curve with boundary {boundary}')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.plot(tpr, fpr)
    plt.show()
    print(f'Thresholds: {thresholds}')

In [5]:
def auc(y_true, y_score, boundary: int):
    y = np.where(y_true > boundary, 0, 1)
    pred = np.where(y_score > boundary, 0, 1)

    return roc_auc_score(y, pred)

In [6]:
scheduler = LearningRateScheduler(schedule)
es = EarlyStopping(monitor='loss', patience=10, verbose=1)
optimizer = Adam(lr=1e-3)

epochs = 1000
validation_freq = 5

In [7]:
start_time = time()
for fname in listdir('model_saves/eval_models'):
    for n in [0, 1, 2, 3, 5, 8]:
        print(f'-------------- Starting {fname.replace(".h5", "")} on noise {n} --------------')
        lstm = load_model(f'model_saves/eval_models/{fname}', compile=False)
        X_train, y_train, X_test, y_test = load_dataset(f'm{n}')

        model = train(dataset=(X_train, y_train, X_test, y_test),
                      model=lstm,
                      epochs=epochs,
                      verbose=0,
                      validation_freq=validation_freq,
                      optimizer=optimizer,
                      callbacks=[scheduler, es])

        for boundary in [300, 500, 1000, 2500, 5000]:
            # plot_roc(y_test, model.predict(X_test), boundary)
            print(auc(y_test, model.predict(X_test), boundary))

        model.save(f'model_saves/evals/{fname.replace(".h5", "")}-{n}.h5')
        print(f'Elapsed time: {time() - start_time}')

-------------- Starting model_0 on noise 0 --------------


KeyboardInterrupt: 